In [1]:
import torch
from torch import nn,optim
import numpy as np
import torchvision 
from torchvision import transforms

In [2]:
batch_size=256
mnist_train=torchvision.datasets.FashionMNIST(
    root='F:\study\ml\DataSet\FashionMNIST',train=True,
    download=True,transform=transforms.ToTensor())
mnist_test=torchvision.datasets.FashionMNIST(
    root='F:\study\ml\DataSet\FashionMNIST',train=False,
    download=True,transform=transforms.ToTensor())

In [3]:
train_iter=torch.utils.data.DataLoader(mnist_train,batch_size,shuffle=True)
test_iter=torch.utils.data.DataLoader(mnist_test,batch_size,shuffle=True)

In [4]:
num_inputs,num_outputs,num_hiddens=784,10,256

In [13]:
class MLPNet(nn.Module):
    def __init__(self,num_inputs,num_hiddens,num_outputs):
        super().__init__()
        self.layer=nn.Sequential(nn.Linear(num_inputs,num_hiddens),
                                nn.ReLU(),
                                nn.Linear(num_hiddens,num_outputs))
#         self.parameters=self.layer.parameters
    def forward(self,x):
        x=x.view(-1,784)
        y=self.layer(x)
        return y
        

In [14]:
net=MLPNet(num_inputs,num_hiddens,num_outputs)
for p in net.parameters():
    print(p.shape)

torch.Size([256, 784])
torch.Size([256])
torch.Size([10, 256])
torch.Size([10])


In [15]:
for n,l in net.named_parameters():
    print(n,l.shape)

layer.0.weight torch.Size([256, 784])
layer.0.bias torch.Size([256])
layer.2.weight torch.Size([10, 256])
layer.2.bias torch.Size([10])


In [17]:
for i in net.named_children():
    print(i)

('layer', Sequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
))


In [19]:
def eval_acc(data_iter):
    acc,n = 0,0
    for x,y in data_iter:
        acc+=(net(x).argmax(dim=1)==y).float().sum()
        n+=len(y)
    return acc/n

In [18]:
lr=0.5
loss=nn.CrossEntropyLoss()
optimizer=optim.SGD(net.parameters(),lr=lr)

In [20]:
for i in range(5):
    train_l,train_acc,test_acc,n=0,0,0,0
    for x,y in train_iter:
        y_hat=net(x)
        l=loss(y_hat,y).sum()
        
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        
        train_l+=l.item()
        train_acc+=(y_hat.argmax(dim=1)==y).float().sum()
        n+=len(y)
    test_acc=eval_acc(test_iter)
    print('epoch %d ,train loss %.3f,train acc %.3f,test acc %.3f' % (
    i+1,train_l/235,train_acc/n,test_acc))

epoch 1 ,train loss 0.787,train acc 0.710,test acc 0.785
epoch 2 ,train loss 0.479,train acc 0.822,test acc 0.830
epoch 3 ,train loss 0.427,train acc 0.842,test acc 0.794
epoch 4 ,train loss 0.388,train acc 0.857,test acc 0.842
epoch 5 ,train loss 0.365,train acc 0.864,test acc 0.838
